In [ ]:
# [예제 3.1] 아파트 매매, 전세 지수 데이터 전처리 시키기
import xlwings as xw
import pandas as pd

def KBpriceindex_preprocessing(path, data_type):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    # data_type : ‘매매종합’, ‘매매APT’, ‘매매연립’, ‘매매단독’, ‘전세종합’, ‘전세APT’, ‘전세연립’, ‘전세단독’ 중 하나
    
    wb = xw.Book(path)                
    sheet = wb.sheets[data_type]   
    row_num = sheet.range(1,1).end('down').end('down').end('down').row  
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
    
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

    for num, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check - 1
                
    big_col[129] = '경기' 
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 12 :
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data


path = r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
price_index = KBpriceindex_preprocessing(path, '매매apt')
jeonse_index = KBpriceindex_preprocessing(path, '전세apt')

In [ ]:
# [예제 3.2] KBpriceindex_preprocessing 함수 실행

import xlwings as xw

def KBpriceindex_preprocessing(path, data_type):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    # data_type : ‘매매종합’, ‘매매APT’, ‘매매연립’, ‘매매단독’, ‘전세종합’, ‘전세APT’, ‘전세연립’, ‘전세단독’ 중 하나
    
    wb = xw.Book(path)                
    sheet = wb.sheets[data_type]   
    row_num = sheet.range(1,1).end('down').end('down').end('down').row  
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
    
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

    for num, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check - 1
                
    big_col[129] = '경기' 
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 20 :
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data

In [ ]:
# [예제 3.3] 아파트 매매가 지수, 전세가 지수 가져오기

import pandas as pd

path = r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
price_index = KBpriceindex_preprocessing(path, '매매apt')
jeonse_index = KBpriceindex_preprocessing(path, '전세apt')

In [ ]:
# [예제 3.4] 매매가 지수 데이터프레임에서 날짜를 기준으로 데이터 가져오기

from datetime import datetime
from dateutil.relativedelta import relativedelta

index_date = datetime(2010, 1, 1)
time_range = 12
prev_date = index_date - relativedelta(months=time_range)

print(index_date)
print(prev_date)

price_index.loc[index_date]

In [ ]:
# [예제 3.5] 1년 동안의 매매가 지수 증감 구하기

(price_index.loc[index_date] - price_index.loc[prev_date])/price_index.loc[prev_date]

In [ ]:
# [예제 3.6] 매매가, 전세가 지수 증감률을 계산해서 데이터프레임 형태로 저장(ch03/ 3.1 부동산 수요.ipynb)

demand_df = pd.DataFrame()
demand_df['매매증감률'] = (price_index.loc[index_date] - price_index.loc[prev_date])/ price_index.loc[prev_date]
demand_df['전세증감률'] = (jeonse_index.loc[index_date] - jeonse_index.loc[prev_date])/jeonse_index.loc[prev_date]

In [ ]:
# [예제 3.7] 기준 날짜로부터 지난 3년간의 데이터 가져오기

prev_date2 = index_date - relativedelta(months=time_range*3)
price_index[prev_date2:index_date][:-1]

In [ ]:
# [예제 3.8] demand_df에 지난 최댓값과 기준 날짜 값의 최댓값 대비 증감률 저장

demand_df['이전최대값'] = price_index[prev_date2:index_date][:-1].max()
demand_df['최댓값대비증감률'] = (price_index.loc[index_date] - demand_df['이전최대값']) /demand_df['이전최대값']

In [ ]:
# [예제 3.9] 매매가 지수와 전세가 지수의 상승여부를 계산해 demand_df에 저장

demand_df['매매가상승'] = demand_df['매매증감률'] > 0.01
demand_df['전세가상승'] = demand_df['전세증감률'] > 0.01

In [ ]:
# [예제 3.10] 전세가 지수 상승이 매매가 지수 상승보다 빠른지와 
# 기준 날짜의 매매가 지수 값이 지난 3년의 최댓값보다 큰지 계산하여 demand_df에 저장

demand_df['더빠른전세상승'] = demand_df['전세증감률'] > demand_df['매매증감률']
demand_df['최댓값대비상승'] = demand_df['최댓값대비증감률'] > 0

In [ ]:
# [예제 3.11] 지역별로 수요 조건의 총합 구하기 (ch03/ 3.1 부동산 수요.ipynb)

demand_df['수요총합'] = demand_df[['매매가상승','전세가상승','더빠른전세상승','최댓값대비상승']].sum(axis=1)

In [ ]:
# [예제 3.12] 수요총합이 4점인 지역만 보기

demand_df = demand_df[demand_df['수요총합'] == 4]

In [ ]:
# [예제 3.13] demand_df에서 원하는 지역만 남기기 

demand_df.loc[[('서울','동대문구'), ('부산', '중구')]]

In [ ]:
# [예제 3.14] 시군 단위의 인덱스만 남기기

seleted_index = []

for name in demand_df.index:
    if name[0] is not name[1]:
        seleted_index.append((name[0], name[1]))
        
demand_df = demand_df.loc[seleted_index]

In [ ]:
# [예제 3.15] 코드를 정리해 함수화 하기 

def demand(price_index, jeonse_index, index_date, time_range):

    prev_date = index_date - relativedelta(months=time_range)
    prev_date2 = index_date - relativedelta(months=time_range*3)

    demand_df = pd.DataFrame()
    demand_df['매매증감률'] = (price_index.loc[index_date] - price_index.loc[prev_date])/ price_index.loc[prev_date].replace(0,None)
    demand_df['전세증감률'] = (jeonse_index.loc[index_date] - jeonse_index.loc[prev_date])/jeonse_index.loc[prev_date].replace(0,None)
    demand_df['이전최대값'] = price_index[prev_date2:index_date][:-1].max()
    demand_df['최댓값대비증감률'] = (price_index.loc[index_date] - demand_df['이전최대값']) /demand_df['이전최대값'].replace(0,None)

    demand_df['매매가상승'] = demand_df['매매증감률'] > 0.01
    demand_df['전세가상승'] = demand_df['전세증감률'] > 0.01
    demand_df['더빠른전세상승'] = demand_df['전세증감률'] > demand_df['매매증감률']
    demand_df['최댓값대비상승'] = demand_df['최댓값대비증감률'] > 0
    demand_df['수요총합'] = demand_df[['매매가상승','전세가상승','더빠른전세상승','최댓값대비상승']].sum(axis=1)

    demand_df = demand_df[demand_df['수요총합'] == 4]

    seleted_index = []

    for name in demand_df.index:
        if name[0] is not name[1]:
            seleted_index.append((name[0], name[1]))

    demand_df = demand_df.loc[seleted_index]
    
    return demand_df

In [ ]:
# [예제 3.16] demand 함수 사용 예시

path = r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
price_index = KBpriceindex_preprocessing(path, '매매apt')
jeonse_index = KBpriceindex_preprocessing(path, '전세apt')

index_date = datetime(2010, 1, 1)
time_range = 12

demand_ex = demand(price_index, jeonse_index, index_date, time_range)

In [ ]:
# [예제 3.17] matplotlib를 불러오고 한글폰트 

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
style.use('ggplot')
%matplotlib inline

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# 맥OS인 경우 위 두 줄을 입력하지 말고 아래 코드를 입력하세요
# rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# [예제 3.18] 특정 날짜를 기준으로 부산 중구 그래프 그리기 

si = '부산'
gu = '중구'
index_date = datetime(2010, 1, 1)

prev_date = index_date - relativedelta(months=12)
prev_date2 = index_date - relativedelta(months=36)
graph_start = index_date - relativedelta(years=3)
graph_end = index_date + relativedelta(years=3)

plt.figure(figsize=(10, 5))
plt.title(si + ' - ' + gu)
plt.plot(price_index[si][gu][graph_start:graph_end], label='매매가')
plt.plot(jeonse_index[si][gu][graph_start:graph_end], label='전세가')
plt.axvline(x=index_date, color='lightcoral', linestyle='--')
plt.axvline(x=prev_date, color='darkseagreen', linestyle='--')
plt.axvline(x=prev_date2, color='darkseagreen', linestyle='--')
plt.legend()
plt.show()

In [ ]:
# [예제 3.19] 수요전략의 결과 데이터프레임에서 지역 이름 가져오기

for name in demand_ex.index:
    print(name)

In [ ]:
# [예제 3.20] demand 함수를 이용해 선택 된 전 지역의 그래프 그리기 

index_date = datetime(2010, 1, 1)

time_range = 12
prev_date = index_date - relativedelta(months=time_range)
prev_date2 = index_date - relativedelta(months=time_range * 3)
graph_start = index_date - relativedelta(months=time_range * 3)
graph_end = index_date + relativedelta(months=time_range * 3)

num_row = int((len(demand_ex.index)-1)/2)+1

plt.figure(figsize=(15, num_row*5))
for i, spot in enumerate(demand_ex.index):
    plt.subplot(num_row, 2, i+1)
    plt.title(spot)
    si = spot[0]
    gu = spot[1]
    plt.plot(price_index[si][gu][graph_start:graph_end], label='매매가')
    plt.plot(jeonse_index[si][gu][graph_start:graph_end], label='전세가')
    plt.axvline(x=index_date, color='lightcoral', linestyle='--')
    plt.axvline(x=prev_date, color='darkseagreen', linestyle='--')
    plt.axvline(x=prev_date2, color='darkseagreen', linestyle='--')
    plt.legend(loc='lower right')
    
plt.show()